# Ceneo Scraper

## Biblioteki

In [15]:
import os
import requests
from bs4 import BeautifulSoup
import json

## Wysłanie żądania dostępu do zasobu do serwera

In [16]:
product_id = "114700014"
url = f"https://www.ceneo.pl/{product_id}/opinie-2"

In [17]:
all_opinions = []
while url:
    response = requests.get(url)
    page_dom = BeautifulSoup(response.text, "html.parser")
    opinions = page_dom.select("div.js_product-review")

    for opinion in opinions:
        try:
            single_opinion = {
                "Opinion_ID": opinion["data-entry-id"],
                "Author": opinion.select_one('span.user-post__author-name').get_text().strip(),
                "Recommendation": opinion.select_one('span.user-post__author-recomendation > em').get_text().strip(),
                "Ratings": opinion.select_one('span.user-post__score-count').get_text().strip(),
                "Content": opinion.select_one('div.user-post__text').get_text().strip(),
                "Pros": [p.get_text().strip() for p in opinion.select('div.review-feature__title--positives ~ div.review-feature__item')],
                "Cons": [c.get_text().strip() for c in opinion.select('div.review-feature__title--negatives ~ div.review-feature__item')],
                "useful": opinion.select_one('span[id^="votes-yes"]').get_text().strip(),
                "useless": opinion.select_one('span[id^="votes-no"]').get_text().strip(),
                "post_date": opinion.select_one('span.user-post__published > time:nth-child(1)')["datetime"].strip(),
                "purchase_date": opinion.select_one('span.user-post__published > time:nth-child(2)')["datetime"].strip()
            }
            all_opinions.append(single_opinion)
        except (AttributeError,TypeError):
            pass
    try:
        url = "https://www.ceneo.pl" + page_dom.select_one("a.pagination__next")["href"]
    except TypeError:
        url = None

## Ekstrakcja składowych pojedynczej opinii
|Składowa|Selektor|Zmienna|
|--------|--------|-------|
|ID opinii|data-entry-id |Opinion ID| 
|Autor|user-post__author-name |Author| 
|Rekomendacja|span.user-post__author-recomendation > em |Reccomendation| 
|gwiazdki|span.user-post__score|Ratings| 
|treść|div.user-post__text|Content| 
|Lista zalet|div.review-feature__title--positive ~ div.review-feature-item|Pros| 
|Lista wad|div.review-feature__title--negatives ~ div.review-feature__item|Cons| 
|Dla ilu przydatna|span[id^="votes-yes"]|useful| 
|Dla ilu nieprzydatna|span[id^="votes-no"]|uselles| 
|Data wystawienia|span.user-post__published > time:nth-child(1)["datetime"]|post date| 
|Data zakupu|span.user-post__published > time:nth-child(2)["datetime"]|purchase date| 

In [18]:
if not os.path.exists("opinions"):
    os.mkdir("opinions")
with open (f"opinions/{product_id}.json", "w", encoding ="UTF-8") as jf:
    json.dump(all_opinions, jf, indent = 4, ensure_ascii=False)

In [19]:
len(all_opinions)

164